In [1]:
# importing the required libraries
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [10]:
# importing the training data from the training file already present in EMNIST dataset
training = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = True, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 50, shuffle = True)
    # importing the testing data
testing = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = False, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 1000, shuffle = True)   
    # creating the neural network class. this class will consist of 3 convolutional layers, 3 max pooling layers and 2 linear layers for now. these layers will be connected by the widely used ReLU activation function. we will be using the Log softmax function for the output layer. these parameters will be varied during the subsequent runs to improve the accuracy
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
    #         input is 1X28X28, output is 26 letters
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 3, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.conv3 = nn.Conv2d(20, 30, kernel_size = 4, stride = 1, padding = 2)
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(30 * 4 * 4, 150)
        self.fc2 = nn.Linear(150, 27)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(-1, 30 * 4 *4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
            # defining the relationship between input and output sizes for usage afterwards
def outputsize(in_size,kernel_size,stride,padding):
    output = int((in_size-kernel_size+2*(padding)) / stride) + 1
    return output
    # defining the various hyperparameters
epochs = 3 
log_interval = 50
network = Net()
optimizer = optim.SGD(network.parameters(), lr = 0.01, momentum = 0.21)
    # creating counters for our results i.e for training losses, test losses, no. of sample trained and no.of samples tested
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(training.dataset) for i in range(epochs+1)]
        # defining the training network
def train(epoch):
    network.train()
    for batch_idx, (data,target) in enumerate(training):
    #       making all the gradients zero before the start of each epoch
        optimizer.zero_grad()
    #     input output and backpropagation steps along with the application of NLL loss function for now
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    #         printing the results after each mini batch has been trained during a particular epoch
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(training.dataset), 100. *batch_idx / len(training), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*100) + ((epoch-1)*len(training.dataset)))
            torch.save(network.state_dict(), '/results/model.pth')
            torch.save(optimizer.state_dict(), '/results/optimizer.pth')
    # defining the testing network
def test():
    network.eval()
    test_loss = 0
    correct = 0
    losstest = []
    #   calculating the loss for each sample and the accuracy
    with torch.no_grad():
        for data, target in testing:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average = False).item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    #         initialising the various parameters to calculate accuracy and print it
        test_loss /= len(testing.dataset)
        losstest.append(test_loss)
        print('\nTest set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testing.dataset), 100. *correct / len(testing.dataset)))
    # initializing the entire network with the datasets and using the above functions to calculate the accuracy
    # first checking the accuracy with random guesses(network uninitialised)
test()
for epoch in range(1,epochs+1):
    train(epoch)
    test()

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. Loss: 3.2990, Accuracy: 796/20800 (3%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.312088
Train Epoch: 1 [2500/124800 (2%)]	Loss: 3.304320
Train Epoch: 1 [5000/124800 (4%)]	Loss: 3.259950
Train Epoch: 1 [7500/124800 (6%)]	Loss: 3.264279
Train Epoch: 1 [10000/124800 (8%)]	Loss: 3.238531
Train Epoch: 1 [12500/124800 (10%)]	Loss: 3.224043
Train Epoch: 1 [15000/124800 (12%)]	Loss: 3.048221
Train Epoch: 1 [17500/124800 (14%)]	Loss: 2.623239
Train Epoch: 1 [20000/124800 (16%)]	Loss: 2.226000
Train Epoch: 1 [22500/124800 (18%)]	Loss: 1.857546
Train Epoch: 1 [25000/124800 (20%)]	Loss: 1.768518
Train Epoch: 1 [27500/124800 (22%)]	Loss: 1.326069
Train Epoch: 1 [30000/124800 (24%)]	Loss: 1.533984
Train Epoch: 1 [32500/124800 (26%)]	Loss: 1.419057
Train Epoch: 1 [35000/124800 (28%)]	Loss: 1.107064
Train Epoch: 1 [37500/124800 (30%)]	Loss: 1.031472
Train Epoch: 1 [40000/124800 (32%)]	Loss: 1.156525
Train Epoch: 1 [42500/124800 (34%)]	Loss: 0.982761
Train Epoch: 1 [45000/124800 (36%)]	Lo

In [11]:
# importing the training data from the training file already present in EMNIST dataset
training = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = True, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 50, shuffle = True)
    # importing the testing data
testing = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = False, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 1000, shuffle = True)   
    # creating the neural network class. this class will consist of 3 convolutional layers, 3 max pooling layers and 2 linear layers for now. these layers will be connected by the widely used ReLU activation function. we will be using the Log softmax function for the output layer. these parameters will be varied during the subsequent runs to improve the accuracy
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
    #         input is 1X28X28, output is 26 letters
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 4, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.conv3 = nn.Conv2d(20, 30, kernel_size = 3, stride = 1, padding = 2)
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(30 * 5 * 5, 150)
        self.fc2 = nn.Linear(150, 27)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(-1, 30 * 5 *5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
            # defining the relationship between input and output sizes for usage afterwards
def outputsize(in_size,kernel_size,stride,padding):
    output = int((in_size-kernel_size+2*(padding)) / stride) + 1
    return output
    # defining the various hyperparameters
epochs = 3 
log_interval = 50
network = Net()
optimizer = optim.SGD(network.parameters(), lr = 0.01, momentum = 0.21)
    # creating counters for our results i.e for training losses, test losses, no. of sample trained and no.of samples tested
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(training.dataset) for i in range(epochs+1)]
        # defining the training network
def train(epoch):
    network.train()
    for batch_idx, (data,target) in enumerate(training):
    #       making all the gradients zero before the start of each epoch
        optimizer.zero_grad()
    #     input output and backpropagation steps along with the application of NLL loss function for now
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    #         printing the results after each mini batch has been trained during a particular epoch
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(training.dataset), 100. *batch_idx / len(training), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*100) + ((epoch-1)*len(training.dataset)))
            torch.save(network.state_dict(), '/results/model.pth')
            torch.save(optimizer.state_dict(), '/results/optimizer.pth')
    # defining the testing network
def test():
    network.eval()
    test_loss = 0
    correct = 0
    losstest = []
    #   calculating the loss for each sample and the accuracy
    with torch.no_grad():
        for data, target in testing:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average = False).item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    #         initialising the various parameters to calculate accuracy and print it
        test_loss /= len(testing.dataset)
        losstest.append(test_loss)
        print('\nTest set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testing.dataset), 100. *correct / len(testing.dataset)))
    # initializing the entire network with the datasets and using the above functions to calculate the accuracy
    # first checking the accuracy with random guesses(network uninitialised)
test()
for epoch in range(1,epochs+1):
    train(epoch)
    test()

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. Loss: 3.2986, Accuracy: 775/20800 (3%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.307807
Train Epoch: 1 [2500/124800 (2%)]	Loss: 3.279421
Train Epoch: 1 [5000/124800 (4%)]	Loss: 3.254948
Train Epoch: 1 [7500/124800 (6%)]	Loss: 3.192298
Train Epoch: 1 [10000/124800 (8%)]	Loss: 3.069177
Train Epoch: 1 [12500/124800 (10%)]	Loss: 2.214749
Train Epoch: 1 [15000/124800 (12%)]	Loss: 1.872674
Train Epoch: 1 [17500/124800 (14%)]	Loss: 2.109681
Train Epoch: 1 [20000/124800 (16%)]	Loss: 1.762794
Train Epoch: 1 [22500/124800 (18%)]	Loss: 1.466552
Train Epoch: 1 [25000/124800 (20%)]	Loss: 1.275283
Train Epoch: 1 [27500/124800 (22%)]	Loss: 1.312887
Train Epoch: 1 [30000/124800 (24%)]	Loss: 0.918740
Train Epoch: 1 [32500/124800 (26%)]	Loss: 1.052560
Train Epoch: 1 [35000/124800 (28%)]	Loss: 0.951508
Train Epoch: 1 [37500/124800 (30%)]	Loss: 0.879073
Train Epoch: 1 [40000/124800 (32%)]	Loss: 0.970002
Train Epoch: 1 [42500/124800 (34%)]	Loss: 0.617345
Train Epoch: 1 [45000/124800 (36%)]	Lo

In [12]:
# importing the training data from the training file already present in EMNIST dataset
training = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = True, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 50, shuffle = True)
    # importing the testing data
testing = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = False, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 1000, shuffle = True)   
    # creating the neural network class. this class will consist of 3 convolutional layers, 3 max pooling layers and 2 linear layers for now. these layers will be connected by the widely used ReLU activation function. we will be using the Log softmax function for the output layer. these parameters will be varied during the subsequent runs to improve the accuracy
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
    #         input is 1X28X28, output is 26 letters
        self.conv1 = nn.Conv2d(1, 10, kernel_size=4, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 3, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.conv3 = nn.Conv2d(20, 30, kernel_size = 3, stride = 1, padding = 2)
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(30 * 5 * 5, 150)
        self.fc2 = nn.Linear(150, 27)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(-1, 30 * 5 *5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
            # defining the relationship between input and output sizes for usage afterwards
def outputsize(in_size,kernel_size,stride,padding):
    output = int((in_size-kernel_size+2*(padding)) / stride) + 1
    return output
    # defining the various hyperparameters
epochs = 3 
log_interval = 50
network = Net()
optimizer = optim.SGD(network.parameters(), lr = 0.01, momentum = 0.21)
    # creating counters for our results i.e for training losses, test losses, no. of sample trained and no.of samples tested
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(training.dataset) for i in range(epochs+1)]
        # defining the training network
def train(epoch):
    network.train()
    for batch_idx, (data,target) in enumerate(training):
    #       making all the gradients zero before the start of each epoch
        optimizer.zero_grad()
    #     input output and backpropagation steps along with the application of NLL loss function for now
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    #         printing the results after each mini batch has been trained during a particular epoch
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(training.dataset), 100. *batch_idx / len(training), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*100) + ((epoch-1)*len(training.dataset)))
            torch.save(network.state_dict(), '/results/model.pth')
            torch.save(optimizer.state_dict(), '/results/optimizer.pth')
    # defining the testing network
def test():
    network.eval()
    test_loss = 0
    correct = 0
    losstest = []
    #   calculating the loss for each sample and the accuracy
    with torch.no_grad():
        for data, target in testing:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average = False).item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    #         initialising the various parameters to calculate accuracy and print it
        test_loss /= len(testing.dataset)
        losstest.append(test_loss)
        print('\nTest set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testing.dataset), 100. *correct / len(testing.dataset)))
    # initializing the entire network with the datasets and using the above functions to calculate the accuracy
    # first checking the accuracy with random guesses(network uninitialised)
test()
for epoch in range(1,epochs+1):
    train(epoch)
    test()

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. Loss: 3.2961, Accuracy: 789/20800 (3%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.297979
Train Epoch: 1 [2500/124800 (2%)]	Loss: 3.296372
Train Epoch: 1 [5000/124800 (4%)]	Loss: 3.287894
Train Epoch: 1 [7500/124800 (6%)]	Loss: 3.282295
Train Epoch: 1 [10000/124800 (8%)]	Loss: 3.283067
Train Epoch: 1 [12500/124800 (10%)]	Loss: 3.267782
Train Epoch: 1 [15000/124800 (12%)]	Loss: 3.256880
Train Epoch: 1 [17500/124800 (14%)]	Loss: 3.222885
Train Epoch: 1 [20000/124800 (16%)]	Loss: 3.172037
Train Epoch: 1 [22500/124800 (18%)]	Loss: 2.871690
Train Epoch: 1 [25000/124800 (20%)]	Loss: 2.354845
Train Epoch: 1 [27500/124800 (22%)]	Loss: 2.296590
Train Epoch: 1 [30000/124800 (24%)]	Loss: 1.529847
Train Epoch: 1 [32500/124800 (26%)]	Loss: 1.760337
Train Epoch: 1 [35000/124800 (28%)]	Loss: 1.329441
Train Epoch: 1 [37500/124800 (30%)]	Loss: 1.071763
Train Epoch: 1 [40000/124800 (32%)]	Loss: 0.862050
Train Epoch: 1 [42500/124800 (34%)]	Loss: 1.265859
Train Epoch: 1 [45000/124800 (36%)]	Lo

In [13]:
# importing the training data from the training file already present in EMNIST dataset
training = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = True, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 50, shuffle = True)
    # importing the testing data
testing = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = False, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 1000, shuffle = True)   
    # creating the neural network class. this class will consist of 3 convolutional layers, 3 max pooling layers and 2 linear layers for now. these layers will be connected by the widely used ReLU activation function. we will be using the Log softmax function for the output layer. these parameters will be varied during the subsequent runs to improve the accuracy
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
    #         input is 1X28X28, output is 26 letters
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 4, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.conv3 = nn.Conv2d(20, 30, kernel_size = 4, stride = 1, padding = 2)
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(30 * 4 * 4, 150)
        self.fc2 = nn.Linear(150, 27)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(-1, 30 * 4 *4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
            # defining the relationship between input and output sizes for usage afterwards
def outputsize(in_size,kernel_size,stride,padding):
    output = int((in_size-kernel_size+2*(padding)) / stride) + 1
    return output
    # defining the various hyperparameters
epochs = 3 
log_interval = 50
network = Net()
optimizer = optim.SGD(network.parameters(), lr = 0.01, momentum = 0.21)
    # creating counters for our results i.e for training losses, test losses, no. of sample trained and no.of samples tested
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(training.dataset) for i in range(epochs+1)]
        # defining the training network
def train(epoch):
    network.train()
    for batch_idx, (data,target) in enumerate(training):
    #       making all the gradients zero before the start of each epoch
        optimizer.zero_grad()
    #     input output and backpropagation steps along with the application of NLL loss function for now
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    #         printing the results after each mini batch has been trained during a particular epoch
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(training.dataset), 100. *batch_idx / len(training), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*100) + ((epoch-1)*len(training.dataset)))
            torch.save(network.state_dict(), '/results/model.pth')
            torch.save(optimizer.state_dict(), '/results/optimizer.pth')
    # defining the testing network
def test():
    network.eval()
    test_loss = 0
    correct = 0
    losstest = []
    #   calculating the loss for each sample and the accuracy
    with torch.no_grad():
        for data, target in testing:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average = False).item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    #         initialising the various parameters to calculate accuracy and print it
        test_loss /= len(testing.dataset)
        losstest.append(test_loss)
        print('\nTest set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testing.dataset), 100. *correct / len(testing.dataset)))
    # initializing the entire network with the datasets and using the above functions to calculate the accuracy
    # first checking the accuracy with random guesses(network uninitialised)
test()
for epoch in range(1,epochs+1):
    train(epoch)
    test()

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. Loss: 3.2935, Accuracy: 801/20800 (3%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.294183
Train Epoch: 1 [2500/124800 (2%)]	Loss: 3.296167
Train Epoch: 1 [5000/124800 (4%)]	Loss: 3.288988
Train Epoch: 1 [7500/124800 (6%)]	Loss: 3.284645
Train Epoch: 1 [10000/124800 (8%)]	Loss: 3.272374
Train Epoch: 1 [12500/124800 (10%)]	Loss: 3.266382
Train Epoch: 1 [15000/124800 (12%)]	Loss: 3.236729
Train Epoch: 1 [17500/124800 (14%)]	Loss: 3.247952
Train Epoch: 1 [20000/124800 (16%)]	Loss: 3.229108
Train Epoch: 1 [22500/124800 (18%)]	Loss: 3.084645
Train Epoch: 1 [25000/124800 (20%)]	Loss: 2.461247
Train Epoch: 1 [27500/124800 (22%)]	Loss: 2.134232
Train Epoch: 1 [30000/124800 (24%)]	Loss: 2.039891
Train Epoch: 1 [32500/124800 (26%)]	Loss: 1.731627
Train Epoch: 1 [35000/124800 (28%)]	Loss: 1.643221
Train Epoch: 1 [37500/124800 (30%)]	Loss: 1.174082
Train Epoch: 1 [40000/124800 (32%)]	Loss: 1.498331
Train Epoch: 1 [42500/124800 (34%)]	Loss: 0.979125
Train Epoch: 1 [45000/124800 (36%)]	Lo

In [14]:
# importing the training data from the training file already present in EMNIST dataset
training = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = True, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 50, shuffle = True)
    # importing the testing data
testing = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = False, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 1000, shuffle = True)   
    # creating the neural network class. this class will consist of 3 convolutional layers, 3 max pooling layers and 2 linear layers for now. these layers will be connected by the widely used ReLU activation function. we will be using the Log softmax function for the output layer. these parameters will be varied during the subsequent runs to improve the accuracy
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
    #         input is 1X28X28, output is 26 letters
        self.conv1 = nn.Conv2d(1, 10, kernel_size=4, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 4, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.conv3 = nn.Conv2d(20, 30, kernel_size = 3, stride = 1, padding = 2)
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(30 * 4 * 4, 150)
        self.fc2 = nn.Linear(150, 27)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(-1, 30 * 4 *4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
            # defining the relationship between input and output sizes for usage afterwards
def outputsize(in_size,kernel_size,stride,padding):
    output = int((in_size-kernel_size+2*(padding)) / stride) + 1
    return output
    # defining the various hyperparameters
epochs = 3 
log_interval = 50
network = Net()
optimizer = optim.SGD(network.parameters(), lr = 0.01, momentum = 0.21)
    # creating counters for our results i.e for training losses, test losses, no. of sample trained and no.of samples tested
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(training.dataset) for i in range(epochs+1)]
        # defining the training network
def train(epoch):
    network.train()
    for batch_idx, (data,target) in enumerate(training):
    #       making all the gradients zero before the start of each epoch
        optimizer.zero_grad()
    #     input output and backpropagation steps along with the application of NLL loss function for now
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    #         printing the results after each mini batch has been trained during a particular epoch
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(training.dataset), 100. *batch_idx / len(training), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*100) + ((epoch-1)*len(training.dataset)))
            torch.save(network.state_dict(), '/results/model.pth')
            torch.save(optimizer.state_dict(), '/results/optimizer.pth')
    # defining the testing network
def test():
    network.eval()
    test_loss = 0
    correct = 0
    losstest = []
    #   calculating the loss for each sample and the accuracy
    with torch.no_grad():
        for data, target in testing:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average = False).item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    #         initialising the various parameters to calculate accuracy and print it
        test_loss /= len(testing.dataset)
        losstest.append(test_loss)
        print('\nTest set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testing.dataset), 100. *correct / len(testing.dataset)))
    # initializing the entire network with the datasets and using the above functions to calculate the accuracy
    # first checking the accuracy with random guesses(network uninitialised)
test()
for epoch in range(1,epochs+1):
    train(epoch)
    test()

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. Loss: 3.2982, Accuracy: 734/20800 (3%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.310565
Train Epoch: 1 [2500/124800 (2%)]	Loss: 3.295566
Train Epoch: 1 [5000/124800 (4%)]	Loss: 3.290817
Train Epoch: 1 [7500/124800 (6%)]	Loss: 3.265923
Train Epoch: 1 [10000/124800 (8%)]	Loss: 3.242192
Train Epoch: 1 [12500/124800 (10%)]	Loss: 3.231559
Train Epoch: 1 [15000/124800 (12%)]	Loss: 3.128833
Train Epoch: 1 [17500/124800 (14%)]	Loss: 2.599334
Train Epoch: 1 [20000/124800 (16%)]	Loss: 2.124582
Train Epoch: 1 [22500/124800 (18%)]	Loss: 1.920908
Train Epoch: 1 [25000/124800 (20%)]	Loss: 1.760690
Train Epoch: 1 [27500/124800 (22%)]	Loss: 1.557215
Train Epoch: 1 [30000/124800 (24%)]	Loss: 1.622482
Train Epoch: 1 [32500/124800 (26%)]	Loss: 1.329498
Train Epoch: 1 [35000/124800 (28%)]	Loss: 1.266629
Train Epoch: 1 [37500/124800 (30%)]	Loss: 1.368212
Train Epoch: 1 [40000/124800 (32%)]	Loss: 0.860116
Train Epoch: 1 [42500/124800 (34%)]	Loss: 1.225881
Train Epoch: 1 [45000/124800 (36%)]	Lo

In [15]:
# importing the training data from the training file already present in EMNIST dataset
training = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = True, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 50, shuffle = True)
    # importing the testing data
testing = torch.utils.data.DataLoader(torchvision.datasets.EMNIST(root = '/results/', split = 'letters' , train = False, download = True, transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,),(0.3081,))])),batch_size = 1000, shuffle = True)   
    # creating the neural network class. this class will consist of 3 convolutional layers, 3 max pooling layers and 2 linear layers for now. these layers will be connected by the widely used ReLU activation function. we will be using the Log softmax function for the output layer. these parameters will be varied during the subsequent runs to improve the accuracy
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
    #         input is 1X28X28, output is 26 letters
        self.conv1 = nn.Conv2d(1, 10, kernel_size=4, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 3, stride = 1, padding = 2)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.conv3 = nn.Conv2d(20, 30, kernel_size = 4, stride = 1, padding = 2)
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        self.fc1 = nn.Linear(30 * 4 * 4, 150)
        self.fc2 = nn.Linear(150, 27)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.view(-1, 30 * 4 *4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)
            # defining the relationship between input and output sizes for usage afterwards
def outputsize(in_size,kernel_size,stride,padding):
    output = int((in_size-kernel_size+2*(padding)) / stride) + 1
    return output
    # defining the various hyperparameters
epochs = 3 
log_interval = 50
network = Net()
optimizer = optim.SGD(network.parameters(), lr = 0.01, momentum = 0.21)
    # creating counters for our results i.e for training losses, test losses, no. of sample trained and no.of samples tested
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(training.dataset) for i in range(epochs+1)]
        # defining the training network
def train(epoch):
    network.train()
    for batch_idx, (data,target) in enumerate(training):
    #       making all the gradients zero before the start of each epoch
        optimizer.zero_grad()
    #     input output and backpropagation steps along with the application of NLL loss function for now
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    #         printing the results after each mini batch has been trained during a particular epoch
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(training.dataset), 100. *batch_idx / len(training), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*100) + ((epoch-1)*len(training.dataset)))
            torch.save(network.state_dict(), '/results/model.pth')
            torch.save(optimizer.state_dict(), '/results/optimizer.pth')
    # defining the testing network
def test():
    network.eval()
    test_loss = 0
    correct = 0
    losstest = []
    #   calculating the loss for each sample and the accuracy
    with torch.no_grad():
        for data, target in testing:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average = False).item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    #         initialising the various parameters to calculate accuracy and print it
        test_loss /= len(testing.dataset)
        losstest.append(test_loss)
        print('\nTest set: Avg. Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testing.dataset), 100. *correct / len(testing.dataset)))
    # initializing the entire network with the datasets and using the above functions to calculate the accuracy
    # first checking the accuracy with random guesses(network uninitialised)
test()
for epoch in range(1,epochs+1):
    train(epoch)
    test()

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. Loss: 3.2934, Accuracy: 1003/20800 (4%)

Train Epoch: 1 [0/124800 (0%)]	Loss: 3.305189
Train Epoch: 1 [2500/124800 (2%)]	Loss: 3.288908
Train Epoch: 1 [5000/124800 (4%)]	Loss: 3.276422
Train Epoch: 1 [7500/124800 (6%)]	Loss: 3.279228
Train Epoch: 1 [10000/124800 (8%)]	Loss: 3.286280
Train Epoch: 1 [12500/124800 (10%)]	Loss: 3.280068
Train Epoch: 1 [15000/124800 (12%)]	Loss: 3.260219
Train Epoch: 1 [17500/124800 (14%)]	Loss: 3.232932
Train Epoch: 1 [20000/124800 (16%)]	Loss: 3.236946
Train Epoch: 1 [22500/124800 (18%)]	Loss: 3.142375
Train Epoch: 1 [25000/124800 (20%)]	Loss: 2.906212
Train Epoch: 1 [27500/124800 (22%)]	Loss: 2.329659
Train Epoch: 1 [30000/124800 (24%)]	Loss: 2.211605
Train Epoch: 1 [32500/124800 (26%)]	Loss: 1.483033
Train Epoch: 1 [35000/124800 (28%)]	Loss: 1.324959
Train Epoch: 1 [37500/124800 (30%)]	Loss: 1.471568
Train Epoch: 1 [40000/124800 (32%)]	Loss: 1.071289
Train Epoch: 1 [42500/124800 (34%)]	Loss: 1.388818
Train Epoch: 1 [45000/124800 (36%)]	L